In [ ]:
from keras.models import load_model
import cv2
import numpy as np
import pyttsx3



In [ ]:
# Load the pre-trained model and the face detector
model = load_model('model-017.model')
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open the webcam
source = cv2.VideoCapture(0)

# Dictionaries for labels and colors
labels_dict = {0: 'MASK', 1: 'NO MASK'}
color_dict = {0: (0, 255, 0), 1: (0, 0, 255)}

# Initialize the TTS engine
engine = pyttsx3.init()

while True:
    # Capture frame-by-frame
    ret, img = source.read()
    if not ret:
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Extract and process the face region
        face_img = gray[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 100, 100, 1))
        result = model.predict(reshaped)

        # Determine the label and color
        label = np.argmax(result, axis=1)[0]
        label_text = labels_dict[label]

        # Text-to-Speech Output
        engine.say(label_text)
        engine.runAndWait()

        # Draw rectangle and label on the image
        cv2.rectangle(img, (x, y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(img, (x, y-40), (x+w, y), color_dict[label], -1)
        cv2.putText(img, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Display the resulting frame
    cv2.imshow('LIVE', img)

    # Break the loop on 'Esc' key press
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release resources
cv2.destroyAllWindows()
source.release()

